In [6]:
import mlflow
from mlflow.models import infer_signature
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from itertools import product

# Load dataset (Diabetes dataset for regression)
X, y = datasets.load_diabetes(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")
mlflow.set_experiment("MLflow Multi-Model")

# Define parameter grids for each model
MODEL_CONFIGS = {
    "Random Forest": [
        {"n_estimators": 100, "random_state": 42},
        {"n_estimators": 200, "random_state": 42},
        {"n_estimators": 300, "random_state": 42}
    ],
    "Neural Net": [
        {"hidden_layer_sizes": (100,), "max_iter": 1000, "random_state": 42},
        {"hidden_layer_sizes": (50, 50), "max_iter": 1000, "random_state": 42},
        {"hidden_layer_sizes": (150,), "max_iter": 1000, "random_state": 42}
    ],
    "SVM": [
        {"kernel": 'rbf', "C": 1.0},
        {"kernel": 'linear', "C": 1.0},
        {"kernel": 'poly', "C": 0.5}
    ],
    "Linear Regression": [
        {}  # No hyperparameters to tune for basic Linear Regression
    ]
}

MODEL_CLASSES = {
    "Random Forest": RandomForestRegressor,
    "Neural Net": MLPRegressor,
    "SVM": SVR,
    "Linear Regression": LinearRegression
}

# Iterate over model types and parameter sets
for model_name, param_list in MODEL_CONFIGS.items():
    for i, params in enumerate(param_list):
        with mlflow.start_run(run_name=f"{model_name} - config {i+1}"):
            model = MODEL_CLASSES[model_name](**params)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Metrics
            rmse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)

            mlflow.log_param("model_name", model_name)
            mlflow.log_params(params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
            mlflow.set_tag("Training Info", f"{model_name} config {i+1} on Diabetes dataset")

            signature = infer_signature(X_train, model.predict(X_train))

            model_info = mlflow.sklearn.log_model(
                sk_model=model,
                artifact_path=f"{model_name.replace(' ', '_').lower()}_model_config_{i+1}",
                signature=signature,
                input_example=X_train,
                registered_model_name=f"{model_name.replace(' ', '_').lower()}_model"
            )

            # Load model back for predictions (optional)
            loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)
            predictions = loaded_model.predict(X_test)

            result = pd.DataFrame(X_test, columns=datasets.load_diabetes().feature_names)
            result["actual"] = y_test
            result["predicted"] = predictions
            print(f"\n{model_name} - Config {i+1} Results:\n", result.head())

Successfully registered model 'random_forest_model'.
2025/05/10 16:10:19 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_model, version 1
Created version '1' of model 'random_forest_model'.



Random Forest - Config 1 Results:
         age       sex       bmi        bp        s1        s2        s3  \
0  0.045341 -0.044642 -0.006206 -0.015999  0.125019  0.125198  0.019187   
1  0.092564 -0.044642  0.036907  0.021872 -0.024960 -0.016658  0.000779   
2  0.063504  0.050680 -0.004050 -0.012556  0.103003  0.048790  0.056003   
3  0.096197 -0.044642  0.051996  0.079265  0.054845  0.036577 -0.076536   
4  0.012648  0.050680 -0.020218 -0.002228  0.038334  0.053174 -0.006584   

         s4        s5        s6  actual  predicted  
0  0.034309  0.032432 -0.005220   219.0     144.00  
1 -0.039493 -0.022517 -0.021788    70.0     171.58  
2 -0.002592  0.084492 -0.017646   202.0     150.72  
3  0.141322  0.098648  0.061054   230.0     253.71  
4  0.034309 -0.005142 -0.009362   111.0     107.19  
🏃 View run Random Forest - config 1 at: http://127.0.0.1:8080/#/experiments/246047683366340720/runs/00df06775388410785ab2df4f31043d9
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/2460

Registered model 'random_forest_model' already exists. Creating a new version of this model...
2025/05/10 16:10:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_model, version 2
Created version '2' of model 'random_forest_model'.



Random Forest - Config 2 Results:
         age       sex       bmi        bp        s1        s2        s3  \
0  0.045341 -0.044642 -0.006206 -0.015999  0.125019  0.125198  0.019187   
1  0.092564 -0.044642  0.036907  0.021872 -0.024960 -0.016658  0.000779   
2  0.063504  0.050680 -0.004050 -0.012556  0.103003  0.048790  0.056003   
3  0.096197 -0.044642  0.051996  0.079265  0.054845  0.036577 -0.076536   
4  0.012648  0.050680 -0.020218 -0.002228  0.038334  0.053174 -0.006584   

         s4        s5        s6  actual  predicted  
0  0.034309  0.032432 -0.005220   219.0    142.925  
1 -0.039493 -0.022517 -0.021788    70.0    176.095  
2 -0.002592  0.084492 -0.017646   202.0    152.285  
3  0.141322  0.098648  0.061054   230.0    255.045  
4  0.034309 -0.005142 -0.009362   111.0    108.525  
🏃 View run Random Forest - config 2 at: http://127.0.0.1:8080/#/experiments/246047683366340720/runs/b0954609fdbb4c6a9d2d6c05d18b2e76
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/2460

Registered model 'random_forest_model' already exists. Creating a new version of this model...
2025/05/10 16:10:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_model, version 3
Created version '3' of model 'random_forest_model'.


...


SVM - Config 2 Results:
         age       sex       bmi        bp        s1        s2        s3  \
0  0.045341 -0.044642 -0.006206 -0.015999  0.125019  0.125198  0.019187   
1  0.092564 -0.044642  0.036907  0.021872 -0.024960 -0.016658  0.000779   
2  0.063504  0.050680 -0.004050 -0.012556  0.103003  0.048790  0.056003   
3  0.096197 -0.044642  0.051996  0.079265  0.054845  0.036577 -0.076536   
4  0.012648  0.050680 -0.020218 -0.002228  0.038334  0.053174 -0.006584   

         s4        s5        s6  actual   predicted  
0  0.034309  0.032432 -0.005220   219.0  142.251886  
1 -0.039493 -0.022517 -0.021788    70.0  141.497300  
2 -0.002592  0.084492 -0.017646   202.0  141.990106  
3  0.141322  0.098648  0.061054   230.0  145.044132  
4  0.034309 -0.005142 -0.009362   111.0  141.554575  
🏃 View run SVM - config 2 at: http://127.0.0.1:8080/#/experiments/246047683366340720/runs/02323ad0a00e4f728457cc74ca2bdef1
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/246047683366340720

Registered model 'svm_model' already exists. Creating a new version of this model...
2025/05/10 16:10:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: svm_model, version 3
Created version '3' of model 'svm_model'.



SVM - Config 3 Results:
         age       sex       bmi        bp        s1        s2        s3  \
0  0.045341 -0.044642 -0.006206 -0.015999  0.125019  0.125198  0.019187   
1  0.092564 -0.044642  0.036907  0.021872 -0.024960 -0.016658  0.000779   
2  0.063504  0.050680 -0.004050 -0.012556  0.103003  0.048790  0.056003   
3  0.096197 -0.044642  0.051996  0.079265  0.054845  0.036577 -0.076536   
4  0.012648  0.050680 -0.020218 -0.002228  0.038334  0.053174 -0.006584   

         s4        s5        s6  actual   predicted  
0  0.034309  0.032432 -0.005220   219.0  144.122444  
1 -0.039493 -0.022517 -0.021788    70.0  140.652680  
2 -0.002592  0.084492 -0.017646   202.0  142.880619  
3  0.141322  0.098648  0.061054   230.0  238.139158  
4  0.034309 -0.005142 -0.009362   111.0  139.436370  
🏃 View run SVM - config 3 at: http://127.0.0.1:8080/#/experiments/246047683366340720/runs/62b6b910a6734d00a08efd91da0c10b6
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/246047683366340720

Successfully registered model 'linear_regression_model'.
2025/05/10 16:10:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: linear_regression_model, version 1
Created version '1' of model 'linear_regression_model'.



Linear Regression - Config 1 Results:
         age       sex       bmi        bp        s1        s2        s3  \
0  0.045341 -0.044642 -0.006206 -0.015999  0.125019  0.125198  0.019187   
1  0.092564 -0.044642  0.036907  0.021872 -0.024960 -0.016658  0.000779   
2  0.063504  0.050680 -0.004050 -0.012556  0.103003  0.048790  0.056003   
3  0.096197 -0.044642  0.051996  0.079265  0.054845  0.036577 -0.076536   
4  0.012648  0.050680 -0.020218 -0.002228  0.038334  0.053174 -0.006584   

         s4        s5        s6  actual   predicted  
0  0.034309  0.032432 -0.005220   219.0  139.547558  
1 -0.039493 -0.022517 -0.021788    70.0  179.517208  
2 -0.002592  0.084492 -0.017646   202.0  134.038756  
3  0.141322  0.098648  0.061054   230.0  291.417029  
4  0.034309 -0.005142 -0.009362   111.0  123.789659  
🏃 View run Linear Regression - config 1 at: http://127.0.0.1:8080/#/experiments/246047683366340720/runs/7e5a5df6cd8142c5abb6a84a9902dd3d
🧪 View experiment at: http://127.0.0.1:8080/#/ex